In [4]:
# import required packages
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [5]:
# Initialize image data generator with rescaling
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)

In [6]:
# Preprocess all test images
train_generator = train_data_gen.flow_from_directory(
        'data/train',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 28709 images belonging to 7 classes.


In [7]:
# Preprocess all train images
validation_generator = validation_data_gen.flow_from_directory(
        'data/test',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 7178 images belonging to 7 classes.


In [8]:
# create model structure
emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))
  
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))
 
emotion_model.add(Flatten())

#fully connected neural networks 
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

cv2.ocl.setUseOpenCL(False)

#Compliling the model  
emotion_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001, decay=1e-6), metrics=['accuracy'])

In [9]:
# display a summary of the architecture and parameters of a model
emotion_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 44, 44, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 22, 22, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 22, 22, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 20, 20, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 10, 10, 128)      0         
 2D)                                                    

In [6]:
# Train the neural network/model
emotion_model_info = emotion_model.fit(
        train_generator,
        steps_per_epoch=28709 // 64,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=7178 // 64)

Epoch 1/50
448/448 [==============================] - 262s 582ms/step - loss: 1.8066 - accuracy: 0.2543 - val_loss: 1.7246 - val_accuracy: 0.3401
Epoch 2/50
448/448 [==============================] - 174s 388ms/step - loss: 1.6417 - accuracy: 0.3612 - val_loss: 1.5502 - val_accuracy: 0.4141
Epoch 3/50
448/448 [==============================] - 179s 400ms/step - loss: 1.5306 - accuracy: 0.4135 - val_loss: 1.4543 - val_accuracy: 0.4464
Epoch 4/50
448/448 [==============================] - 183s 409ms/step - loss: 1.4543 - accuracy: 0.4427 - val_loss: 1.3921 - val_accuracy: 0.4749
Epoch 5/50
448/448 [==============================] - 953s 2s/step - loss: 1.3924 - accuracy: 0.4710 - val_loss: 1.3479 - val_accuracy: 0.4859
Epoch 6/50
448/448 [==============================] - 219s 488ms/step - loss: 1.3418 - accuracy: 0.4930 - val_loss: 1.3013 - val_accuracy: 0.5033
Epoch 7/50
448/448 [==============================] - 212s 473ms/step - loss: 1.2980 - accuracy: 0.5063 - val_loss: 1.2723 - va

In [14]:
# save model structure in json file
# model_json = emotion_model.to_json()
# with open("emotion_model.json", "w") as json_file:
#     json_file.write(model_json)

# # save trained model weight in .h5 file
# emotion_model.save_weights('emotion_model.h5')

In [16]:
import os

# Define the file path or directory path
path_to = "E:\\CS\\Emotion_Detection\\model"

# Create the directory if it doesn't exist
os.makedirs(path_to, exist_ok=True)

# Construct the complete file path by joining path_to and fer_json
file_path = os.path.join(path_to, "emotion_model.json")

# Write to the JSON file
with open(file_path, "w") as json_file:
    json_file.write(model_json)

# Store the model weights to fer.h5
emotion_model.save_weights(os.path.join(path_to, "emotion_model.h5"))